In [1]:
import scipy.io
import numpy as np
import random
import os

In [2]:
person_nums = [1,2,4,5,6,8,9,10,11,12,13,17,19,21,22,25,26,27,28,29]
## Build the baseline model for emotion recognition with dropout layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, SimpleRNN, LSTM, Conv2D, Flatten, MaxPooling2D, GRU, AveragePooling2D, Dropout, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
def individual_model(features_list):
    input_layers = []
    hidden_layers = []
    combined_feature = np.empty((len(features_list[0]),0))
    for i, feature in enumerate(features_list):
        
        if len(feature.shape) == 3:
            input_i = Input(shape=(feature.shape[1], feature.shape[2]))
            input_layers.append(input_i)

            hidden_i = input_i[:,:,:,None]
            hidden_i = Conv2D(32, 3, activation='relu')(hidden_i)
            hidden_i = AveragePooling2D((3, 3))(hidden_i)
            hidden_i = Dropout(0.5)(hidden_i)
            hidden_i = Conv2D(16, 3, activation='relu')(hidden_i)
            hidden_i = AveragePooling2D((3, 3))(hidden_i)
            hidden_i = Dropout(0.5)(hidden_i)
            hidden_i = Conv2D(8, 1, activation='relu')(hidden_i)
            hidden_i = AveragePooling2D((2, 2))(hidden_i)
            hidden_i = Dropout(0.5)(hidden_i)
            hidden_i = Conv2D(4, 1, activation='relu')(hidden_i)
            hidden_i = AveragePooling2D((2, 2))(hidden_i)
            hidden_i = Dropout(0.5)(hidden_i)
            hidden_i = Flatten()(hidden_i)

            hidden_layers.append(hidden_i)
        elif feature.shape[1] <20:
            combined_feature = np.hstack((combined_feature, feature))
            
        else:  # For series features
            input_i = Input(shape=(feature.shape[1],))
            input_layers.append(input_i)
            hidden_i = Lambda(lambda x: x[:, :, None])(input_i)  # Add a new dimension
            hidden_i = LSTM(4)(hidden_i)
            hidden_layers.append(hidden_i)
    input_i = Input(shape=(combined_feature.shape[1],))
    input_layers.append(input_i)
    dense_num = np.max((1, int(combined_feature.shape[1]/2)))
    hidden_i = Dense(dense_num, activation='relu')(input_i)
    hidden_layers.append(hidden_i)
    print(combined_feature.shape)
    concat_layer = concatenate(hidden_layers)
    h = Dropout(0.2)(concat_layer)
    h = Dense(64, activation='relu')(h)
    h = Dense(32, activation='relu')(h)
    output_layer = Dense(2)(h)
    model = Model(inputs=input_layers, outputs=output_layer)
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

import tensorflow as tf
from tensorflow.keras.callbacks import Callback
import numpy as np

class PruningCallback(tf.keras.callbacks.Callback):
    def __init__(self, initial_pruning_factor=0.1, final_pruning_factor=0.5, start_epoch=0, end_epoch=None, frequency=1):
        super(PruningCallback, self).__init__()
        self.initial_pruning_factor = initial_pruning_factor
        self.final_pruning_factor = final_pruning_factor
        self.start_epoch = start_epoch
        self.end_epoch = end_epoch if end_epoch is not None else np.inf
        self.frequency = frequency
        self.pruned_weights = {}
        self.layer_importance = {}

    def get_pruning_factor(self, epoch):
        if epoch < self.start_epoch:
            return 0
        if epoch > self.end_epoch:
            return self.final_pruning_factor
        return self.initial_pruning_factor + (self.final_pruning_factor - self.initial_pruning_factor) * (epoch - self.start_epoch) / (self.end_epoch - self.start_epoch)

    def on_train_begin(self, logs=None):
        total_weight_magnitude = 0
        for layer in self.model.layers:
            if hasattr(layer, 'get_weights'):
                weights = layer.get_weights()
                layer_norm = sum(np.linalg.norm(w) for w in weights)
                total_weight_magnitude += layer_norm
                self.layer_importance[layer.name] = layer_norm
    
        # Normalize the layer importance values so they sum up to 1
        for layer_name in self.layer_importance:
            self.layer_importance[layer_name] /= total_weight_magnitude
    # def on_train_begin(self, logs=None):
    #     total_weight_magnitude = sum([np.linalg.norm(layer.get_weights()) for layer in self.model.layers if hasattr(layer, 'get_weights')])
    #     for layer in self.model.layers:
    #         if hasattr(layer, 'get_weights'):
    #             self.layer_importance[layer.name] = np.linalg.norm(layer.get_weights()) / total_weight_magnitude

    def get_layer_pruning_factor(self, layer_name, global_pruning_factor):
        if layer_name in self.layer_importance:
            importance = self.layer_importance[layer_name]
            adjusted_pruning_factor = global_pruning_factor * (1 - importance)
            return min(adjusted_pruning_factor, 1)  # Ensure the pruning factor is not greater than 1
        return global_pruning_factor
    def prune_weights(self, layer, global_pruning_factor):
        
        weights = layer.get_weights()
        layer_name = layer.name
        pruning_factor = self.get_layer_pruning_factor(layer_name, global_pruning_factor)

        if layer_name not in self.pruned_weights:
            self.pruned_weights[layer_name] = [np.zeros_like(w, dtype=bool) for w in weights]
        
        for i in range(len(weights)):
            weight = weights[i]
            # print(weight.shape)
            # print(weight.size)
            if weight.ndim > 1:  # Only prune dense or convolutional layers
                unpruned_weights = np.logical_not(self.pruned_weights[layer_name][i])
                num_unpruned = np.sum(unpruned_weights)
                num_pruning = min(num_unpruned, int(weight.size * pruning_factor) - np.sum(self.pruned_weights[layer_name][i]))
                num_pruning = int(weight.size * pruning_factor) - np.sum(self.pruned_weights[layer_name][i])
                if num_pruning > 0:
                    unpruned_flat_indices = np.flatnonzero(unpruned_weights)
                    abs_unpruned_weights = np.abs(weight[unpruned_weights])
                    pruning_flat_indices = np.argpartition(abs_unpruned_weights, num_pruning)[:num_pruning]
                    
                    indices = np.unravel_index(pruning_flat_indices, weight.shape)
                    self.pruned_weights[layer_name][i][indices] = True

                weights[i] = weights[i]*(~self.pruned_weights[layer_name][i])
                
        layer.set_weights(weights)

    def on_epoch_end(self, epoch, logs=None):
        if (epoch - self.start_epoch) % self.frequency != 0:
            return

        pruning_factor = self.get_pruning_factor(epoch)
        for layer in self.model.layers:
            if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
                self.prune_weights(layer, pruning_factor)
from feature_emotion import feature_extract, split_data, label_unique_tuples, split_data_unknown, split_data_few_shot
gts, sensor_nums, walk_nums, trace_nums, people_nums, spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features = feature_extract(person_nums)

walk_nums_all = np.squeeze(walk_nums)
trace_nums_all = np.squeeze(trace_nums)
people_nums_all = np.squeeze(people_nums)

## 0: train, 1: validation 2: test

test_person_id = [10]
ra_all = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
idx = 30
for ra in ra_all:
    flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
    ## Data Normalization before training ans testing
    import tensorflow as tf
    tf.compat.v1.disable_v2_behavior()
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
    from sklearn.preprocessing import StandardScaler
    scalers = []
    X_train_normalized = []
    X_val_normalized = []
    X_test_normalized = []
    train_idx = np.where(flag_tr_val_te ==0)[0]
    np.random.shuffle(train_idx)
    val_idx = np.where(flag_tr_val_te ==1)[0]
    test_idx = np.where(flag_tr_val_te ==2)[0]
    
    for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
        scaler = StandardScaler()
        if len(feature.shape)==2:
            X_train_i = feature[train_idx,:]
            X_val_i = feature[val_idx,:]
            X_test_i = feature[test_idx,:]
            X_train_normalized_i = scaler.fit_transform(X_train_i)
            X_val_normalized_i = scaler.transform(X_val_i)
            X_test_normalized_i = scaler.transform(X_test_i)
            scalers.append(scaler)
        else:
            X_train_i = feature[train_idx,:,:]
            X_val_i = feature[val_idx,:,:]
            X_test_i = feature[test_idx,:,:]
            X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
            X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
            X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
            scalers.append(scaler)
        X_train_normalized.append(X_train_normalized_i)
        X_val_normalized.append(X_val_normalized_i)
        X_test_normalized.append(X_test_normalized_i)
    y_train = gts[train_idx,:]
    y_val = gts[val_idx,:]
    y_test = gts[test_idx,:]
    X_train_normalized_new = []
    combined_feature = np.empty((len(X_train_normalized[0]),0))
    for feature in X_train_normalized:
        if len(feature.shape) == 3:
            X_train_normalized_new.append(feature)
        elif feature.shape[1] <20:
            combined_feature = np.hstack((combined_feature, feature))
        else:
            X_train_normalized_new.append(feature)
    X_train_normalized_new.append(combined_feature)
    
    X_val_normalized_new = []
    combined_feature = np.empty((len(X_val_normalized[0]),0))
    for feature in X_val_normalized:
        if len(feature.shape) == 3:
            X_val_normalized_new.append(feature)
        elif feature.shape[1] <20:
            combined_feature = np.hstack((combined_feature, feature))
        else:
            X_val_normalized_new.append(feature)
    X_val_normalized_new.append(combined_feature)
    
    X_test_normalized_new = []
    combined_feature = np.empty((len(X_test_normalized[0]),0))
    for feature in X_test_normalized:
        if len(feature.shape) == 3:
            X_test_normalized_new.append(feature)
        elif feature.shape[1] <20:
            combined_feature = np.hstack((combined_feature, feature))
        else:
            X_test_normalized_new.append(feature)
    X_test_normalized_new.append(combined_feature)
    
    
    num_epochs = 50
    
    
    # Gradually prune weights in dense and convolutional layers from 10% to 50% over the course of training, starting from epoch 0.
    
    
    rates = [0.4, 0.5, 0.6]
    
    for r in rates:
        model = individual_model(X_train_normalized)
        model.compile(loss='mean_absolute_error', optimizer='adam')
        model_name = './checkpoints/unknown_person_few_shot_baseline_p'+str(test_person_id[0])+'_'+str(idx)+'.h5'
        checkpoint = ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    
        pruning_callback = PruningCallback(initial_pruning_factor=0.01, final_pruning_factor=r, start_epoch=5, end_epoch=20, frequency=1)
        model.fit(X_train_normalized_new, y_train, epochs=num_epochs, batch_size=1024, validation_data=(X_val_normalized_new, y_val), callbacks=[pruning_callback, checkpoint])
        import tensorflow as tf
        model = tf.keras.models.load_model(model_name)
        from tensorflow.keras.models import Model
        layers = model.layers
        second_last_layer_output = layers[-4].output
        feature_extractor_model = Model(inputs=model.input, outputs=second_last_layer_output)
        train_features = feature_extractor_model.predict(X_train_normalized_new)
        test_features = feature_extractor_model.predict(X_test_normalized_new)
        
        p_train = people_nums[train_idx,:]
        p_val = people_nums[val_idx,:]
        p_test = people_nums[test_idx,:]
        ## Calculate the distance between test person and training person
        def euclidean_distance(a, b):
            return np.sqrt(np.sum((a - b) ** 2, axis=1))
        
        distance_dict = {}
        for ii in range(len(person_nums)):
            if person_nums[ii] == test_person_id[0]:
                continue
            else:
                
                ind = np.where(p_train ==person_nums[ii])[0]
                tmp_train_features = train_features[ind, :]
                distances = np.array([euclidean_distance(train_sample, test_features) for train_sample in tmp_train_features])
                print(distances.shape)
                average_distances = np.mean(distances, axis=1)
        
                # Step 4: Find the overall average distance
                overall_average_distance = np.mean(average_distances)
                distance_dict[person_nums[ii]] = overall_average_distance
        
        
        def normalize_to_weights(distance_dict):
            distances = np.array(list(distance_dict.values()))
            # Handle the case where a distance is zero to avoid division by zero
            distances = np.clip(distances, a_min=1e-10, a_max=None)
            weights = 1 / distances
            normalized_weights = weights
            # normalized_weights = weights / sum(weights)
            # print(sum(weights))
            # print(sum(normalized_weights))
            # Assign the normalized weights back to the dictionary
            normalized_weight_dict = dict(zip(distance_dict.keys(), normalized_weights))
            return normalized_weight_dict
        def scale_dict_values(my_dict):
            scaled_dict = my_dict.copy()
            min_val = min(scaled_dict.values())
            max_val = max(scaled_dict.values())
            
            for key in scaled_dict:
                scaled_dict[key] = 1 + 9 * (scaled_dict[key] - min_val) / (max_val - min_val)
            
            return scaled_dict
        weights_dict = normalize_to_weights(distance_dict)
        weights_dict = scale_dict_values(weights_dict)
        print(weights_dict)
        
        w_train = np.zeros_like(p_train)
        for i in range(len(w_train)):
            if p_train[i] == test_person_id[0]:
                w_train[i] = 50
            else:
                w_train[i] = weights_dict[int(p_train[i])]
        
        w_train = np.squeeze(w_train)
        
        import sys
        sys.path.append('..')
        # for layer in model.layers[-4:]:
        #     layer.trainable = False
        # model = individual_model.individual_model(X_train_normalized)
        model_name = './checkpoints/unknown_person_few_shot_p'+str(test_person_id[0])+'_'+str(idx)+'.h5'
        checkpoint = ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
        pruning_callback = PruningCallback(initial_pruning_factor=r, final_pruning_factor=r, start_epoch=1, end_epoch=20, frequency=1)
        
        history = model.fit(x=X_train_normalized_new, y=y_train,sample_weight= w_train,epochs=20, batch_size=1024, validation_data=(X_val_normalized_new, y_val), callbacks=[pruning_callback, checkpoint])
        import sys
        sys.path.append('..')
        # for layer in model.layers[-4:]:
        #     layer.trainable = False
        # model = individual_model.individual_model(X_train_normalized)
        model_name = './checkpoints/unknown_person_few_shot_baseline_p'+str(test_person_id[0])+'_'+str(idx)+'.h5'
        model = tf.keras.models.load_model(model_name)
        model_name = './checkpoints/unknown_person_few_shot_baseline_2_p'+str(test_person_id[0])+'_'+str(idx)+'.h5'
        idx += 1
        checkpoint = ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
        pruning_callback = PruningCallback(initial_pruning_factor=r, final_pruning_factor=r, start_epoch=1, end_epoch=20, frequency=1)
        
        history = model.fit(x=X_train_normalized_new, y=y_train,epochs=20, batch_size=1024, validation_data=(X_val_normalized_new, y_val), callbacks=[pruning_callback, checkpoint])

2023-11-22 10:14:59.340879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


35211
Instructions for updating:
non-resource variables are not supported in the long term


2023-11-22 10:16:22.772418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 10:16:22.783850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 10:16:22.784103: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

2023-11-22 10:16:23.814367: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 10:16:23.814608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 10:16:23.814799: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

(31684, 95)
Train on 31684 samples, validate on 3527 samples


...

Epoch 1/20


2023-11-23 01:08:46.945205: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indropout_85_2/cond/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


32573/32573 [==============================] - ETA: 0s - loss: 1.4151

2023-11-23 01:09:13.890824: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_107/mul' id:1014293 op device:{requested: '', assigned: ''} def:{{{node loss_107/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_107/mul/x, loss_107/dense_71_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.



Epoch 1: val_loss improved from inf to 1.38626, saving model to ./checkpoints/unknown_person_few_shot_baseline_2_p10_47.h5
32573/32573 [==============================] - 126s 4ms/sample - loss: 1.4151 - val_loss: 1.3863
Epoch 2/20
32573/32573 [==============================] - ETA: 0s - loss: 1.4104
Epoch 2: val_loss improved from 1.38626 to 1.38128, saving model to ./checkpoints/unknown_person_few_shot_baseline_2_p10_47.h5
32573/32573 [==============================] - 23s 709us/sample - loss: 1.4104 - val_loss: 1.3813
Epoch 3/20
32573/32573 [==============================] - ETA: 0s - loss: 1.4125
Epoch 3: val_loss improved from 1.38128 to 1.38117, saving model to ./checkpoints/unknown_person_few_shot_baseline_2_p10_47.h5
32573/32573 [==============================] - 23s 720us/sample - loss: 1.4125 - val_loss: 1.3812
Epoch 4/20
32573/32573 [==============================] - ETA: 0s - loss: 1.4124
Epoch 4: val_loss improved from 1.38117 to 1.37867, saving model to ./checkpoints/unkn